Since this project is based on doing [this Kaggle competition](https://www.kaggle.com/c/rsna-pneumonia-detection-challenge), my data gathering wasn't exactly that difficult to accomplish. Downloading files from Kaggle is pretty easy, a 3.3GB zip file that unzipped into ~3.6GB of mostly pictures. 

## Data Description Directly from Kaggle

### What files do I need?

This is a two-stage challenge. You will need the images for the current stage - provided as stage_1_train_images.zip and stage_1_test_images.zip. You will also need the training data - stage_1_train_labels.csv - and the sample submission stage_1_sample_submission.csv, which provides the IDs for the test set, as well as a sample of what your submission should look like. The file stage_1_detailed_class_info.csv contains detailed information about the positive and negative classes in the training set, and may be used to build more nuanced models.
<br>
### What should I expect the data format to be?

The training data is provided as a set of patientIds and bounding boxes. Bounding boxes are defined as follows: x-min y-min width height

There is also a binary target column, Target, indicating pneumonia or non-pneumonia.

There may be multiple rows per patientId.

#### DICOM Images
All provided images are in DICOM format.

<span style='color:green'>(My thoughts) The pictures are all `.dcm` files which don't open in normal picture viewing applications. The files are scans of the person's lungs but have a lot of other metadata (or at least has the ability to hold a lot of other meta data as it is a medically formated patient file) as opposed to a single image. This adds a level of complexity I was not expecting. I've found an application to open the files but you have to pay for the full version. The trial version allows you to use 10k files and I have over 20k so that isn't really enough. I'm looking into other applications that will open the files. Along with that, I'm not entirely sure how to open the image itself in python. This will require some more thought.</span>
<br>
### What am I predicting?
In this challenge competitors are predicting whether pneumonia exists in a given image. They do so by predicting bounding boxes around areas of the lung. Samples without bounding boxes are negative and contain no definitive evidence of pneumonia. Samples with bounding boxes indicate evidence of pneumonia.

When making predictions, competitors should predict as many bounding boxes as they feel are necessary, in the format: confidence x-min y-min width height

There should be only ONE predicted row per image. This row may include multiple bounding boxes.

A properly formatted row may look like any of the following.

For patientIds with no predicted pneumonia / bounding boxes: 0004cfab-14fd-4e49-80ba-63a80b6bddd6,

For patientIds with a single predicted bounding box: 0004cfab-14fd-4e49-80ba-63a80b6bddd6,0.5 0 0 100 100

For patientIds with multiple predicted bounding boxes: 0004cfab-14fd-4e49-80ba-63a80b6bddd6,0.5 0 0 100 100 0.5 0 0 100 100, etc.

### File descriptions
 - stage_1_train.csv - the training set. Contains patientIds and bounding box / target information.
 - stage_1_sample_submission.csv - a sample submission file in the correct format. Contains patientIds for the test set. Note that the sample submission contains one box per image, but there is no limit to the number of bounding boxes that can be assigned to a given image.
 - stage_1_detailed_class_info.csv - provides detailed information about the type of positive or negative class for each image.

### Data fields
 - patientId _- A patientId. Each patientId corresponds to a unique image.
 - x_ - the upper-left x coordinate of the bounding box.
 - y_ - the upper-left y coordinate of the bounding box.
 - width_ - the width of the bounding box.
 - height_ - the height of the bounding box.
 - Target_ - the binary Target, indicating whether this sample has evidence of pneumonia.

In [4]:
import pandas as pd
import numpy as np
import pandas_profiling as pp

#### stage_1_detailed_class_info

In [2]:
stage_1_detailed_class_info = pd.read_csv('./data/stage_1_detailed_class_info.csv', )

In [6]:
stage_1_detailed_class_info.shape

(28989, 2)

In [8]:
stage_1_detailed_class_info.describe()

,patientId,class
count,28989,28989
unique,25684,3
top,3239951b-6211-4290-b237-3d9ad17176db,No Lung Opacity / Not Normal
freq,4,11500


In [9]:
stage_1_detailed_class_info.head(5)

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity


In [10]:
stage_1_detailed_class_info.isnull().sum()

patientId    0
class        0
dtype: int64

In [17]:
stage_1_detailed_class_info['class'].value_counts()

No Lung Opacity / Not Normal    11500
Lung Opacity                     8964
Normal                           8525
Name: class, dtype: int64

In [11]:
# Strange that there is a problem with this ..
pp.ProfileReport(stage_1_detailed_class_info)

/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3124: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-0.5, right=-0.5
  'left=%s, right=%s') % (left, right))
/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3443: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.5, top=-0.5
  'bottom=%s, top=%s') % (bottom, top))


ZeroDivisionError: float division by zero

#### stage_1_train_labels


In [12]:
stage_1_train_labels = pd.read_csv('./data/stage_1_train_labels.csv')

In [13]:
stage_1_train_labels.shape

(28989, 6)

In [14]:
stage_1_train_labels.describe()

,x,y,width,height,Target
count,8964.000000,8964.000000,8964.000000,8964.000000,28989.000000
mean,391.456158,363.135877,220.845382,334.174364,0.309221
std,203.945378,148.607149,59.041384,158.097239,0.462180
min,2.000000,2.000000,40.000000,45.000000,0.000000
25%,205.000000,246.000000,180.000000,207.000000,0.000000
50%,320.000000,360.000000,219.000000,304.000000,0.000000
75%,591.000000,475.000000,261.000000,445.000000,1.000000
max,817.000000,881.000000,528.000000,942.000000,1.000000


In [15]:
stage_1_train_labels.isnull().sum()
#There are a number of nulls here. I'm guessing that they are the ones without any pneumonia


patientId        0
x            20025
y            20025
width        20025
height       20025
Target           0
dtype: int64

In [18]:
pp.ProfileReport(stage_1_train_labels)

Number of variables,6
Number of observations,28989
Total Missing (%),46.1%
Total size in memory,1.3 MiB
Average record size in memory,48.0 B
Numeric,4
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


#### Sample Submission


In [19]:
sample_submission = pd.read_csv('./data/stage_1_sample_submission.csv')

In [20]:
sample_submission.head()

,patientId,PredictionString
0,000924cf-0f8d-42bd-9158-1af53881a557,0.5 0 0 100 100
1,000db696-cf54-4385-b10b-6b16fbb3f985,0.5 0 0 100 100
2,000fe35a-2649-43d4-b027-e67796d412e0,0.5 0 0 100 100
3,001031d9-f904-4a23-b3e5-2c088acd19c6,0.5 0 0 100 100
4,0010f549-b242-4e94-87a8-57d79de215fc,0.5 0 0 100 100


In [21]:
sample_submission.shape

(1000, 2)

In [37]:
sample_submission.isnull().sum()

patientId           0
PredictionString    0
dtype: int64

In [43]:
sample_submission['PredictionString'].value_counts() #-_-

0.5 0 0 100 100    1000
Name: PredictionString, dtype: int64

In [22]:
pp.ProfileReport(sample_submission)

/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3124: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-0.5, right=-0.5
  'left=%s, right=%s') % (left, right))
/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_base.py:3443: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.5, top=-0.5
  'bottom=%s, top=%s') % (bottom, top))


ZeroDivisionError: float division by zero

#### Credits

In [35]:
credits = open('./data/GCP Credits Request Link - RSNA.txt', 'r')

In [36]:
print(credits.read())

https://www.kaggle.com/GCP-Credits-Form-RSNA-Pneumonia

